In [1]:
import os
import numpy as np
import pandas as pd
from loaders.loading import load_train
from transformers.dash_transformer import DashTransformer
from transformers.dict_nbhd_transformer import DictNBHDTransformer
from transformers.dict_transformer import DictTransformer
from transformers.digit_transformer import DigitTransformer
from transformers.flat_transformer import FlatTransformer
from transformers.dict_class_transformer import DictClassTransformer
from sklearn.pipeline import Pipeline
from transformers.letters_transformer import LettersTransformer
from transformers.scored_chain import ScoredChain
from transformers.self_transformer import SelfTransformer
from time import gmtime, strftime
from transformers.simple_transliterator import SimpleTransliterator

In [2]:
df = load_train(['class', 'before', 'after'], r'../input/norm_challenge_ru')
df['before_prev'] = df['before'].shift(1)
df['before_next'] = df['before'].shift(-1)
df = df.fillna('')
print(df.info())

classes = ['PLAIN', 'DATE', 'PUNCT', 'ORDINAL', 'VERBATIM', 'LETTERS', 'CARDINAL', 'MEASURE', 'TELEPHONE', 'ELECTRONIC',
           'DECIMAL', 'DIGIT', 'FRACTION', 'MONEY', 'TIME']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10574516 entries, 0 to 10574515
Data columns (total 5 columns):
class          object
before         object
after          object
before_prev    object
before_next    object
dtypes: object(5)
memory usage: 403.4+ MB
None


In [3]:
test_size = int(len(df.index)*0.1)
x_train = df.drop(['after'], axis=1).head(len(df.index)-test_size)
y_train = df['after'].head(len(df.index)-test_size)
x_test = df.drop(['after'], axis=1).tail(test_size)
y_test = df['after'].tail(test_size)

In [4]:
transform_chain = ScoredChain([
    ('dash', DashTransformer()),
    ('digit', DigitTransformer()),
    ('punсt', DictClassTransformer(u'PUNCT', 1.0)),
    ('verbatim', DictClassTransformer(u'VERBATIM', 1.0)),
    ('self', SelfTransformer(threshold=0.5, modelpath='models/self.model.train_9517064_0.00117_0.3_500_6')),
    ('dict_nbhd', DictNBHDTransformer(0.5)),
    ('letters', LettersTransformer()),
    # ('dict', DictTransformer(0.5)),
    ('translit', SimpleTransliterator('cyrtranslit')),
    # ('flat', FlatTransformer())
], metrics=[
    ('tp', lambda X: np.mean((X["after"] == y_test))),
    ('fp', lambda X: np.mean((X["after"].notnull()) & ~(X["after"] == y_test)))
])
predicted_val = transform_chain.fit(x_train, y_train)
predicted = transform_chain.transform(x_test)
tp = np.mean((predicted["after"] == y_test))
fp = np.mean((predicted["after"].notnull()) & ~(predicted["after"] == y_test))
print(f'pipeline test TP {tp}')
print(f'pipeline test FP {fp}')

MorphologyExtractor create index: 0it [00:00, ?it/s]

MorphologyExtractor create index: 10918it [00:00, 109065.43it/s]

MorphologyExtractor create index: 14194it [00:00, 60870.61it/s] 

MorphologyExtractor create index: 17288it [00:00, 51862.34it/s]

MorphologyExtractor create index: 22712it [00:00, 52410.77it/s]

...

SimpleTransliterator transform:  90%|█████████ | 955263/1057451 [00:02<00:00, 471031.69it/s]

SimpleTransliterator transform:  95%|█████████▍| 1001031/1057451 [00:02<00:00, 466701.56it/s]

SimpleTransliterator transform:  99%|█████████▉| 1044814/1057451 [00:02<00:00, 464725.54it/s]

SimpleTransliterator transform: 100%|██████████| 1057451/1057451 [00:02<00:00, 463191.57it/s]

pipeline test TP 0.9324952172724789
pipeline test FP 0.036643778293273165


In [5]:
predicted[(predicted["after"] == y_test)]['class'].value_counts()

PLAIN         697132
PUNCT         229395
CARDINAL       15800
VERBATIM       15627
LETTERS        15403
DATE            8469
ORDINAL         2300
MEASURE         1181
DIGIT            183
ELECTRONIC       181
TIME             154
TELEPHONE        130
DECIMAL           76
FRACTION          29
MONEY              8
Name: class, dtype: int64

In [6]:
predicted[~(predicted["after"] == y_test) & ~(predicted["after"].isnull())]['class'].value_counts()

PLAIN         33480
LETTERS        3205
CARDINAL        769
ELECTRONIC      380
ORDINAL         371
DATE            329
MEASURE         116
VERBATIM         41
MONEY            33
TELEPHONE        12
DECIMAL           8
TIME              4
FRACTION          1
Name: class, dtype: int64

In [7]:
predicted[(predicted["after"].isnull())]['class'].value_counts()

CARDINAL      11139
DATE          10201
PLAIN          3898
MEASURE        2628
ORDINAL        2170
TELEPHONE       861
DECIMAL         649
LETTERS         426
MONEY           246
FRACTION        227
TIME            125
ELECTRONIC       64
Name: class, dtype: int64

In [ ]:
predicted[~(predicted["after"] == y_test) & (predicted["after"].isnull()) & (predicted['class']=='PLAIN')].assign(after_true=y_test).sample(20)

,class,before,before_prev,before_next,after,after_true
9928198,CARDINAL,13,.,Акт,None,тринадцать
9730953,CARDINAL,4165,(,человек,None,четыре тысячи сто шестьдесят пять
9751544,CARDINAL,538,от,—,None,пятисот тридцати восьми
9979915,CARDINAL,21,-,"""",None,двадцать один
9675008,CARDINAL,173,сложности,ученика,None,ста семидесяти трех
9622166,CARDINAL,2002,наций,",",None,две тысячи два
10134458,CARDINAL,605,сделано,выстрелов,None,шестьсот пять
10566962,CARDINAL,1916,«,»,None,тысяча девятьсот шестнадцать
9945427,CARDINAL,30,примерно,омматидиев,None,тридцати
10463680,CARDINAL,135,типа,.,None,сто тридцать пять


In [17]:
df[~(df['before'].str.match('^-?\d*\s?\d*\s?\/\s?\d+\s?\d*$')) & (df['class'] == 'FRACTION')]

,class,before,after,before_prev,before_next
40420,FRACTION,7½,семи целых и одной второй,около,фунтов
157446,FRACTION,3½,три целых и одну вторую,на,очка
445580,FRACTION,59½,пятидесяти девяти целых и одной второй,139 976 рублей,копейки
765884,FRACTION,¼,одна четвертая,Второй,финалДата
803808,FRACTION,½,одна вторая,номиналом,шиллинга
999135,FRACTION,⅔,две третьих,на,территории
1253476,FRACTION,½,одна вторая,CrA,gut
1584453,FRACTION,8½,восемь целых и одна вторая,«,женщин
1584466,FRACTION,8½,восемь целых и одна вторая,.,Women
1876892,FRACTION,½,одна вторая,в,","
